In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_train = pd.read_excel('processminer-rareevent.xlsx')

In [ ]:
# CLEAN TRAIN DATA WITH CYCLE

x_cols = raw_train.drop(['y'],axis = 1)
y_col = raw_train['y']

a = []
b = []
c = []
for i in range(len(raw_train)):
    if i < 2:
        continue
    else:
        a.append(y_col.values[i])
        b.append(x_cols.values[i-1] - x_cols.values[i-2])
        c.append(x_cols.values[i-1] - x_cols.values[i-3])
        
cl_1 = pd.DataFrame(a)
cl_2 = pd.DataFrame(b)
cl_3 = pd.DataFrame(c)

clean_train_data = pd.concat([cl_1,cl_2,cl_3],axis = 1)

In [ ]:
writer = pd.ExcelWriter('without_breaks.xlsx')
clean_data_nn.to_excel(writer,'Sheet1',index=False)
writer.save()

In [5]:
cols=['x%d' % i for i in range(1, 123, 1)]
cols.insert(0,'y')
clean_train_data.columns=[cols]

In [12]:
X_train = clean_train_data.iloc[:,1:123].values
y_train = clean_train_data['y'].values

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_train_1, y_train_1 = sm.fit_sample(X_train, y_train.ravel())
print(X_train_1.shape)

(32440, 122)


In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_tran = sc.fit_transform(X_train_1)

In [19]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()
classifier.add(Dense(122,input_dim =122, activation='relu',kernel_initializer = 'uniform'))
classifier.add(Dense(122,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(122,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(122,activation='relu',kernel_initializer='uniform'))
classifier.add(Dense(2,activation='softmax',kernel_initializer='uniform'))
classifier.compile( optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
classifier.fit(X_train_tran,y_train_1,epochs=9,batch_size = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/9
32440/32440 [==============================] - 6s 197us/step - loss: 0.1005 - acc: 0.9567
Epoch 2/9
32440/32440 [==============================] - 4s 115us/step - loss: 0.0124 - acc: 0.9965
Epoch 3/9
32440/32440 [==============================] - 4s 108us/step - loss: 0.0110 - acc: 0.9970
Epoch 4/9
32440/32440 [==============================] - 3s 106us/step - loss: 0.0058 - acc: 0.9983
Epoch 5/9
32440/32440 [==============================] - 3s 99us/step - loss: 0.0042 - acc: 0.9986
Epoch 6/9
32440/32440 [==============================] - 3s 102us/step - loss: 0.0091 - acc: 0.9978
Epoch 7/9
32440/32440 [==============================] - 3s 103us/step - loss: 0.0045 - acc: 0.9987
Epoch 8/9
32440/32440 [==============================] - 3s 99us/step - loss: 8.6222e-04 - acc: 0.9998
Epoch 9/9
32440/32440 [==============================] - 3s 105us/step - loss: 0.0034 - acc: 0.9990 0s - loss: 0.0036 - acc:


In [30]:
raw_test = pd.read_excel('only_test.xlsx')

In [37]:
# CLEAN TEST DATA WITH CYCLE

x_cols = raw_test.drop(['y'],axis = 1)
y_col = raw_test['y']

a = []
b = []
c = []
for i in range(len(raw_test)):
    if i < 2:
        continue
    if raw_train['y'][i] == 0:
        if raw_train['y'][i-1] == 1 or raw_train['y'][i-2] == 1:
            continue
        else:
            a.append(y_col.values[i])
            b.append(x_cols.values[i] - x_cols.values[i-1])
            c.append(x_cols.values[i] - x_cols.values[i-2])
            
    else:
        a.append(y_col.values[i])
        b.append(x_cols.values[i] - x_cols.values[i-1])
        c.append(x_cols.values[i] - x_cols.values[i-2])
        
cl_1 = pd.DataFrame(a)
cl_2 = pd.DataFrame(b)
cl_3 = pd.DataFrame(c)

clean_test_data = pd.concat([cl_1,cl_2,cl_3],axis = 1)

In [38]:
cols=['x%d' % i for i in range(1, 123, 1)]
cols.insert(0,'y')
clean_test_data.columns=[cols]

In [39]:
X_test = clean_test_data.iloc[:,1:123].values
y_test = clean_test_data['y'].values

In [40]:
X_test_tran = sc.transform(X_test)

In [41]:
from sklearn.metrics import confusion_matrix,classification_report

predictions=classifier.predict(X_test_tran)
predictions_classes=classifier.predict_classes(X_test_tran)
print(confusion_matrix(y_test,predictions_classes))
print(classification_report(y_test,predictions_classes))

[[1791    5]
 [   8    4]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1796
           1       0.44      0.33      0.38        12

   micro avg       0.99      0.99      0.99      1808
   macro avg       0.72      0.67      0.69      1808
weighted avg       0.99      0.99      0.99      1808

